# Handwritten Digit Classfication using Bidirectional Recurrent Neural Network

In this example, we are going to use the MNIST dataset to train a multi-layer feed foward neural network. MNIST is a simple computer vision dataset of handwritten digits. It has 60,000 training examles and 10,000 test examples. "It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting." For more details, please checkout the website [MNIST](http://yann.lecun.com/exdb/mnist/)

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.utils._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.dataset.DataSet
import com.intel.analytics.bigdl.dataset.image.{BytesToGreyImg, GreyImgNormalizer, GreyImgToBatch, GreyImgToSample}
import com.intel.analytics.bigdl.models.lenet.Utils._
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.optim._
import com.intel.analytics.bigdl.optim.{Adam, Top1Accuracy, Trigger}
import com.intel.analytics.bigdl.visualization.{TrainSummary, ValidationSummary}
import com.intel.analytics.bigdl.tensor.Tensor
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

## 1. Load MNIST dataset

In [2]:
import java.nio.ByteBuffer
import java.nio.file.{Files, Path, Paths}

import com.intel.analytics.bigdl.dataset.ByteRecord
import com.intel.analytics.bigdl.utils.File
import scopt.OptionParser

def load(featureFile: String, labelFile: String): Array[ByteRecord] = {
    val featureBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(featureFile)))
    val labelBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(labelFile)))
    
    val labelMagicNumber = labelBuffer.getInt()
    require(labelMagicNumber == 2049)
    val featureMagicNumber = featureBuffer.getInt()
    require(featureMagicNumber == 2051)

    val labelCount = labelBuffer.getInt()
    val featureCount = featureBuffer.getInt()
    require(labelCount == featureCount)

    val rowNum = featureBuffer.getInt()
    val colNum = featureBuffer.getInt()

    val result = new Array[ByteRecord](featureCount)
    var i = 0
    while (i < featureCount) {
      val img = new Array[Byte]((rowNum * colNum))
      var y = 0
      while (y < rowNum) {
        var x = 0
        while (x < colNum) {
          img(x + y * colNum) = featureBuffer.get()
          x += 1
        }
        y += 1
      }
      result(i) = ByteRecord(img, labelBuffer.get().toFloat + 1.0f)
      i += 1
    }

    result
}

Then we need to set paths of data. Please edit paths if they are changed.

In [3]:
val trainData = "../../datasets/mnist/train-images-idx3-ubyte"
val trainLabel = "../../datasets/mnist/train-labels-idx1-ubyte"
val validationData = "../../datasets/mnist/t10k-images-idx3-ubyte"
val validationLabel = "../../datasets/mnist/t10k-labels-idx1-ubyte"

## 2. Recurent Neural Network Model Setup

Bidirectional RNNs are based on the idea that the output at time t may not only depend on the previous elements in the sequence, but also future elements. They combine an RNN that moves foward through time beginning from the end of the sequence with another RNN that moves backward through time from the end of the sequence.

In [4]:
//Parameters
val batchSize = 64
val maxEpochs = 5

//Network Parameters
val nInput = 28 //MNIST data input (img shape: 28*28)
val nHidden = 128 // hidden layer num of features
val nClasses = 10  //MNIST total classes (0-9 digits)

Then the data set should be created and the model needs to be established.

In [5]:
val trainSet = 
    DataSet.array(load(trainData, trainLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(trainMean, trainStd) -> GreyImgToBatch(batchSize)
val validationSet = 
    DataSet.array(load(validationData, validationLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToBatch(batchSize)

In [6]:
val recurrent = BiRecurrent(JoinTable(3, 3)).add(LSTM(nInput, nHidden))
val rnnModel = Sequential().add(InferReshape(Array(-1, nInput), true)).add(recurrent).add(Select(2, -1)).add(Linear(2*nHidden, nClasses))

## 3. Optimizer Setup

In [7]:
val optimizer = Optimizer(model = rnnModel, dataset = trainSet, criterion = CrossEntropyCriterion[Float]())
optimizer.setValidation(trigger = Trigger.everyEpoch, dataset = validationSet, vMethods = Array(new Top1Accuracy))
optimizer.setOptimMethod(new Adam())
optimizer.setEndWhen(Trigger.maxEpoch(maxEpochs))

com.intel.analytics.bigdl.optim.DistriOptimizer@783bcb1f

The following is to create training and validation summary.

In [8]:
import java.text.SimpleDateFormat
import java.util.Calendar
val today = Calendar.getInstance
val formatDate = new SimpleDateFormat("yyyyMMdd-hhmmss")
val name = "rnn-" + formatDate.format(today.getTime()).toString()
val trainSummary = TrainSummary(logDir="/tmp/bigdl_summaries", appName=name)
trainSummary.setSummaryTrigger("Parameters", Trigger.severalIteration(50))
val valSummary = ValidationSummary(logDir="/tmp/bigdl_summaries", appName=name)
optimizer.setTrainSummary(trainSummary)
optimizer.setValidationSummary(valSummary)
printf("saving logs to %s", name)

saving logs to rnn-20170922-012336

In [9]:
// Boot training process
val trainedModel = optimizer.optimize()
print("Optimization Done.")

can't find locality partition for partition 0 Partition locations are (ArrayBuffer(172.168.2.109)) Candidate partition locations are
(0,List()).
Optimization Done.

In [10]:
val rddData = sc.parallelize(load(validationData, validationLabel), batchSize)
val transformer = BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToSample()
val evaluationSet = transformer(rddData)
        
val result = trainedModel.evaluate(evaluationSet, Array(new Top1Accuracy[Float]), Some(batchSize))

result.foreach(r => println(s"${r._2} is ${r._1}"))

Top1Accuracy is Accuracy(correct: 9818, count: 10000, accuracy: 0.9818)


In [11]:
val predictions = trainedModel.predict(evaluationSet)
val preLables = predictions.take(8).map(_.toTensor.max(1)._2.valueAt(1)).mkString(",")
val lables = evaluationSet.take(8).map(_.label.valueAt(1)).mkString(",")
println(preLables)
println(lables)

5.0,2.0,5.0,10.0,5.0,2.0,5.0,10.0
8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0


## 4. Draw the performance curve

In [12]:
import vegas._
import vegas.render.HTMLRenderer._
import vegas.DSL._

In [13]:
val loss = trainSummary.readScalar("Loss")
val top1 = valSummary.readScalar("Top1Accuracy")

val lossXY = loss.map(_ ._1).zip(loss.map(_ ._2)).toSeq

Vegas(description = "The Loss curve.", width = 700.0, height = 300.0).
  withXY(lossXY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, title = "Loss").
  mark(Line).
  show

val top1XY = top1.map(_ ._1).zip(top1.map(_ ._2)).toSeq
Vegas(description = "The Top1Accuracy curve.", width = 700.0, height = 300.0).
  withXY(top1XY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, bin = Bin(base = 0.9), title = "Top1Accuracy").
  mark(Line).
  show

<iframe id="frame-vegas-dc4db0da-e129-4f35-b7f2-eff4e215b6f5" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-dc4db0da-e129-4f35-b7f2-eff4e215b6f5'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "title" : "Loss"
 }
 },
 "description" : "The Loss curve.",
 "data" : {
 "values" : [
 {
 "x" : 1,
 "y" : 2.3027944564819336
 },
 {
 "x" : 2,
 "y" : 2.2938292026519775
 },
 {
 "x" : 3,
 "y" : 2.299825668334961
 },
 {
 "x" : 4,
 "y" : 2.306598663330078
 },
 {
 "x" : 5,
 "y" : 2.28338885307312
 },
 {
 "x" : 6,
 "y" : 2.278653621673584
 },
 {
 "x" : 7,
 "y" : 2.283207416534424
 },
 {
 "x" : 8,
 "y" : 2.265207052230835
 },
 {
 "x" : 9,
 "y" : 2.2664365768432617
 },
 {
 "x" : 10,
 "y" : 2.2528433799743652
 },
 {
 "x" : 11,
 "y" : 2.2656688690185547
 },
 {
 "x" : 12,
 "y" : 2.229377031326294
 },
 {
 "x" : 13,
 "y" : 2.249831438064575
 },
 {
 "x" : 14,
 "y" : 2.219137191772461
 },
 {
 "x" : 15,
 "y" : 2.2346298694610596
 },
 {
 "x" : 16,
 "y" : 2.208136558532715
 },
 {
 "x" : 17,
 "y" : 2.1969876289367676
 },
 {
 "x" : 18,
 "y" : 2.1717257499694824
 },
 {
 "x" : 19,
 "y" : 2.172793388366699
 },
 {
 "x" : 20,
 "y" : 2.163506507873535
 },
 {
 "x" : 21,
 "y" : 2.1254849433898926
 },
 {
 "x" : 22,
 "y" : 2.0369133949279785
 },
 {
 "x" : 23,
 "y" : 2.0343477725982666
 },
 {
 "x" : 24,
 "y" : 2.004728317260742
 },
 {
 "x" : 25,
 "y" : 2.052830457687378
 },
 {
 "x" : 26,
 "y" : 2.086160659790039
 },
 {
 "x" : 27,
 "y" : 2.007821798324585
 },
 {
 "x" : 28,
 "y" : 2.0872435569763184
 },
 {
 "x" : 29,
 "y" : 1.8973133563995361
 },
 {
 "x" : 30,
 "y" : 1.9062879085540771
 },
 {
 "x" : 31,
 "y" : 1.886789083480835
 },
 {
 "x" : 32,
 "y" : 1.8842594623565674
 },
 {
 "x" : 33,
 "y" : 1.9038689136505127
 },
 {
 "x" : 34,
 "y" : 1.7845547199249268
 },
 {
 "x" : 35,
 "y" : 1.885033130645752
 },
 {
 "x" : 36,
 "y" : 1.7935519218444824
 },
 {
 "x" : 37,
 "y" : 1.661949872970581
 },
 {
 "x" : 38,
 "y" : 1.5276775360107422
 },
 {
 "x" : 39,
 "y" : 1.8309426307678223
 },
 {
 "x" : 40,
 "y" : 1.6818697452545166
 },
 {
 "x" : 41,
 "y" : 1.6223816871643066
 },
 {
 "x" : 42,
 "y" : 1.6054346561431885
 },
 {
 "x" : 43,
 "y" : 1.554704189300537
 },
 {
 "x" : 44,
 "y" : 1.4742012023925781
 },
 {
 "x" : 45,
 "y" : 1.6782587766647339
 },
 {
 "x" : 46,
 "y" : 1.6495479345321655
 },
 {
 "x" : 47,
 "y" : 1.4736679792404175
 },
 {
 "x" : 48,
 "y" : 1.3300721645355225
 },
 {
 "x" : 49,
 "y" : 1.6785728931427002
 },
 {
 "x" : 50,
 "y" : 1.4631811380386353
 },
 {
 "x" : 51,
 "y" : 1.264460563659668
 },
 {
 "x" : 52,
 "y" : 1.571540355682373
 },
 {
 "x" : 53,
 "y" : 1.4537912607192993
 },
 {
 "x" : 54,
 "y" : 1.3132596015930176
 },
 {
 "x" : 55,
 "y" : 1.4183244705200195
 },
 {
 "x" : 56,
 "y" : 1.3395895957946777
 },
 {
 "x" : 57,
 "y" : 1.329850196838379
 },
 {
 "x" : 58,
 "y" : 1.2330000400543213
 },
 {
 "x" : 59,
 "y" : 1.3536548614501953
 },
 {
 "x" : 60,
 "y" : 1.2916728258132935
 },
 {
 "x" : 61,
 "y" : 1.353787899017334
 },
 {
 "x" : 62,
 "y" : 1.3948663473129272
 },
 {
 "x" : 63,
 "y" : 1.346284031867981
 },
 {
 "x" : 64,
 "y" : 1.3834559917449951
 },
 {
 "x" : 65,
 "y" : 1.2323558330535889
 },
 {
 "x" : 66,
 "y" : 1.2902765274047852
 },
 {
 "x" : 67,
 "y" : 1.178239107131958
 },
 {
 "x" : 68,
 "y" : 1.1

<iframe id="frame-vegas-6576ec29-77a7-4824-9f46-b6f146b0f34c" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-6576ec29-77a7-4824-9f46-b6f146b0f34c'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "bin" : {
 "base" : 0.9
 },
 "title" : "Top1Accuracy"
 }
 },
 "description" : "The Top1Accuracy curve.",
 "data" : {
 "values" : [
 {
 "x" : 938,
 "y" : 0.9621000289916992
 },
 {
 "x" : 1876,
 "y" : 0.9704999923706055
 },
 {
 "x" : 2814,
 "y" : 0.9790999889373779
 },
 {
 "x" : 3752,
 "y" : 0.9817000031471252
 },
 {
 "x" : 4690,
 "y" : 0.9817000031471252
 }
 ]
 }
}
 }
 vg.embed("#vegas-6576ec29-77a7-4824-9f46-b6f146b0f34c", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

Finally, the Spark should be stopped.

In [2]:
sc.stop()